In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Reshape 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.datasets import mnist 

In [ ]:
# Load the MNIST dataset (handwritten digits)
(train_images, _), (_, _) = mnist.load_data()

In [ ]:
# Preprocess the data
train_images = train_images / 255.0  # Normalize pixel values to the range [0, 1]
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)


In [ ]:
# Setting random seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
# Defining the Generator
generator = Sequential([
    Dense(128, input_shape=(100,), activation='relu'),
    Dense(784, activation='sigmoid'),
    Reshape((28, 28, 1))
])

In [ ]:
# Defining the Discriminator
discriminator = Sequential([
    Flatten(input_shape=(28, 28, 1)),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])


In [ ]:
# Compile the Discriminator
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5), metrics=['accuracy'])

In [ ]:
# Combine the Generator and Discriminator to create the GAN
discriminator.trainable = False
gan = Sequential([generator, discriminator])
gan.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))

In [ ]:
# Training loop
epochs = 10000
batch_size = 128

In [ ]:
for epoch in range(epochs):
    noise = np.random.normal(0, 1, (batch_size, 100))
    generated_images = generator.predict(noise)
    real_images = train_images[np.random.randint(0, train_images.shape[0], batch_size)]
    labels_real = np.ones((batch_size, 1))
    labels_fake = np.zeros((batch_size, 1))

    # Train the discriminator
    d_loss_real = discriminator.train_on_batch(real_images, labels_real)
    d_loss_fake = discriminator.train_on_batch(generated_images, labels_fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train the generator
    noise = np.random.normal(0, 1, (batch_size, 100))
    labels_gan = np.ones((batch_size, 1))
    g_loss = gan.train_on_batch(noise, labels_gan)

    if epoch % 100 == 0:
        print(f"Epoch: {epoch}, D Loss: {d_loss[0]}, G Loss: {g_loss}")

    if epoch % 1000 == 0:
        # Save a generated image
        plt.imshow(generated_images[0].reshape(28, 28), cmap='gray')
        plt.axis('off')
        plt.savefig(f"gan_generated_image_epoch_{epoch}.png")
        plt.close()